In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

The first dataset comes from the [U.S. Government’s open data](https://www.data.gov/). Here is the official description of the dataset (can be found [here](https://catalog.data.gov/dataset/disaster-declaration)):
<center>The FEMA Disaster Declarations Summary is a summarized dataset describing all <b>federally declared disasters</b>, starting with the first disaster declaration in <b>1953</b>, featuring all three disaster declaration types: major disaster, emergency and fire management assistance. </center>

In [48]:
df = pd.read_excel("https://www.fema.gov/media-library-data/" +
                   "1552661411911-55ceb500e0427af9f3724fcf916645f0/data.gov.FEMADeclarations.3.15.19.xlsx",
                   sheet_name="FEMA Declarations",
                   usecols=["Declaration Date",
                            "Disaster Number", 
                            "State ", 
                            "Disaster Type", 
                            "Incident Type", 
                            "Title", 
                            "Incident Begin Date", 
                            "Incident End Date"],
                   dtype={"Disaster Type" : 'category', 
                          "Incident Type" : 'category',
                          "State " : 'category'
                         },
                   header=2)
df.rename({"State " : "State"}, inplace=True, axis=1)
df.head()

,Disaster Number,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date
0,4419,AL,2019-03-05 17:33:00,DR,Tornado,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",2019-03-03 00:01:00,2019-03-03 23:59:00
1,4418,WA,2019-03-04 14:38:00,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",2018-12-10 00:00:00,2018-12-24 00:00:00
2,4418,WA,2019-03-04 14:38:00,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",2018-12-10 00:00:00,2018-12-24 00:00:00
3,4418,WA,2019-03-04 14:38:00,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",2018-12-10 00:00:00,2018-12-24 00:00:00
4,4418,WA,2019-03-04 14:38:00,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",2018-12-10 00:00:00,2018-12-24 00:00:00


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48555 entries, 0 to 48554
Data columns (total 8 columns):
Disaster Number        48555 non-null int64
State                  48555 non-null category
Declaration Date       48555 non-null datetime64[ns]
Disaster Type          48555 non-null category
Incident Type          48555 non-null category
Title                  48555 non-null object
Incident Begin Date    48555 non-null datetime64[ns]
Incident End Date      48216 non-null datetime64[ns]
dtypes: category(3), datetime64[ns](3), int64(1), object(1)
memory usage: 1.8+ MB


According to the MetaData sheet into the xlsx file, the **Disaster Number** column is a number used to designate an event declared as a disaster.

In [50]:
disaster_number_counts = df["Disaster Number"].value_counts()
disaster_number_counts

1239    257
3261    255
3216    254
1624    254
1606    254
       ... 
2706      1
2957      1
2674      1
2424      1
1495      1
Name: Disaster Number, Length: 3568, dtype: int64

We see that this number isn't unique. Let's analyse a sample for a non-unique Disaster Number.

In [51]:
index = 0
most_occured_disaster_number = df.loc[disaster_number_counts.index[index] == df["Disaster Number"]]
most_occured_disaster_number.sample(4)

,Disaster Number,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date
31127,1239,TX,1998-08-26 15:50:08,DR,Severe Storm(s),TROPICAL STORM CHARLEY,1998-08-22 13:55:00,1998-08-31 12:00:00
31110,1239,TX,1998-08-26 15:50:08,DR,Severe Storm(s),TROPICAL STORM CHARLEY,1998-08-22 13:55:00,1998-08-31 12:00:00
30929,1239,TX,1998-08-26 15:50:08,DR,Severe Storm(s),TROPICAL STORM CHARLEY,1998-08-22 13:55:00,1998-08-31 12:00:00
31073,1239,TX,1998-08-26 15:50:08,DR,Severe Storm(s),TROPICAL STORM CHARLEY,1998-08-22 13:55:00,1998-08-31 12:00:00


It looks like these rows are redundant. Let's verify it for each column for this given index.

In [52]:
most_occured_disaster_number.apply(lambda col:print(f'{col.name} contains only 1 element : {col.value_counts(normalize=True).iloc[0] == 1.0}'));

Disaster Number contains only 1 element : True
State contains only 1 element : True
Declaration Date contains only 1 element : True
Disaster Type contains only 1 element : True
Incident Type contains only 1 element : True
Title contains only 1 element : True
Incident Begin Date contains only 1 element : True
Incident End Date contains only 1 element : True


All these lines refer to the same event. Consequently, we can drop duplicated without considering a subset of columns.

In [53]:
df.drop_duplicates(subset=None, keep='first', inplace=True)
df.shape

(3570, 8)

Way better ! We can then look after remaining duplicated **Disaster Number**.

In [54]:
disaster_number_counts = df["Disaster Number"].value_counts()
remaining_duplicates = disaster_number_counts[disaster_number_counts != 1]
remaining_duplicates

1110    2
572     2
Name: Disaster Number, dtype: int64

In [55]:
df[df["Disaster Number"].isin(remaining_duplicates.index)]

,Disaster Number,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date
33521,1110,AR,1996-04-23,DR,Severe Storm(s),SEVERE STORMS AND TORNADOES,1996-04-18,1996-04-19
33523,1110,IL,1996-04-23,DR,Severe Storm(s),SEVERE STORMS AND TORNADOES,1996-04-18,1996-04-19
41759,572,MH,1979-02-12,DR,Typhoon,TYPHOON ALICE,1979-02-12,1979-02-12
41767,572,PW,1979-02-12,DR,Typhoon,TYPHOON ALICE,1979-02-12,1979-02-12


Only 2 numbers remaining ! Great ! The only difference is the **State** column. With the help of [wikipedia](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations), we can add a column containing the full name of the state instead of this 2 letters code.

In [56]:
states_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations", 
                         attrs = {'class': 'wikitable sortable'},
                         header = 11)[0]
states_df = states_df.rename({"United States of America" : "Fullname", "US" : "Code"}, axis=1)[["Fullname", "Code"]]
states_df.dropna(inplace=True)
states_df.sample(5)

,Fullname,Code
10,Georgia,GA
30,New Jersey,NJ
52,Guam,GU
44,Utah,UT
18,Louisiana,LA


In [57]:
states_df.shape

(60, 2)

And we can now merge our 2 datasets.

In [63]:
df_with_states = (df.merge(states_df, how='left',left_on='State', right_on="Code")
                  .drop(["Code"], axis=1)
                  .rename({"Fullname":"State name"}, axis=1))
df_with_states.sample(5)

,Disaster Number,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,State name
1012,1759,SD,2008-05-22 16:30:00,DR,Snow,SEVERE WINTER STORM AND RECORD AND NEAR RECORD...,2008-05-01 00:00:00,2008-05-02,South Dakota
1330,2603,OK,2006-01-02 18:05:00,FM,Fire,CASHION FIRE COMPLEX,2006-01-01 00:00:00,NaT,Oklahoma
2123,1287,TX,1999-08-22 21:30:00,DR,Hurricane,HURRICANE BRET,1999-08-21 20:00:00,1999-08-26,Texas
389,3350,MA,2012-10-28 17:35:00,EM,Hurricane,HURRICANE SANDY,2012-10-27 00:00:00,2012-11-08,Massachusetts
2884,609,CA,1979-10-19 00:00:00,DR,Earthquake,EARTHQUAKE,1979-10-19 00:00:00,1979-10-19,California


To be sure, I want to verify we have a **State name** for each disaster.

In [65]:
df_with_states["State name"].isna().sum()

0

Let's look again to our remaining duplicated **Disaster Number**

In [67]:
df_with_states[df_with_states["Disaster Number"].isin(remaining_duplicates.index)]

,Disaster Number,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,State name
2341,1110,AR,1996-04-23,DR,Severe Storm(s),SEVERE STORMS AND TORNADOES,1996-04-18,1996-04-19,Arkansas
2342,1110,IL,1996-04-23,DR,Severe Storm(s),SEVERE STORMS AND TORNADOES,1996-04-18,1996-04-19,Illinois
2929,572,MH,1979-02-12,DR,Typhoon,TYPHOON ALICE,1979-02-12,1979-02-12,Marshall Islands
2930,572,PW,1979-02-12,DR,Typhoon,TYPHOON ALICE,1979-02-12,1979-02-12,Palau


From the [following map](https://images-na.ssl-images-amazon.com/images/I/81r8Hxz5lQL._SL1500_.jpg), we can suppose it is because these events are spread on several states. Indeed, **Arkansas** and **Illinois** are in the same region. The same hypothesis can be done considering [**Palau**](https://en.wikipedia.org/wiki/Palau) & [**Marshall Islands**](https://en.wikipedia.org/wiki/Marshall_Islands).

<img width="600px" src="https://images-na.ssl-images-amazon.com/images/I/81r8Hxz5lQL._SL1500_.jpg">

Let's now consider the **Incident Type** column in order to only consider natural disasters caused by weather.

In [71]:
df_with_states["Incident Type"].value_counts()

Severe Storm(s)     941
Fire                829
Flood               805
Hurricane           353
Tornado             166
Snow                165
Severe Ice Storm     62
Typhoon              61
Drought              46
Earthquake           31
Coastal Storm        27
Other                26
Freezing             18
Toxic Substances      8
Mud/Landslide         7
Volcano               6
Fishing Losses        6
Dam/Levee Break       4
Human Cause           3
Tsunami               3
Terrorist             2
Chemical              1
Name: Incident Type, dtype: int64

In [73]:
weather_disasters = ["Severe Storm(s)", 
                     "Fire", "Flood", 
                     "Hurricane", "Tornado", 
                     "Snow", "Severe Ice Storm", 
                     "Typhoon", "Drought", 
                     "Coastal Storm", "Freezing"]
df_weather_disasters = df_with_states[df_with_states["Incident Type"].isin(weather_disasters)]
df_weather_disasters.shape

(3473, 9)

In [74]:
df_weather_disasters.sample(5)

,Disaster Number,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,State name
1289,1634,TN,2006-04-05 18:59:00,DR,Severe Storm(s),SEVERE STORMS AND TORNADOES,2006-04-02 19:00:00,2006-04-08 00:00:00,Tennessee
95,3391,PR,2017-09-18 18:50:00,EM,Hurricane,HURRICANE MARIA,2017-09-17 17:00:00,2017-11-15 23:59:00,Puerto Rico
740,1922,MT,2010-07-10 10:00:00,DR,Severe Storm(s),SEVERE STORMS AND FLOODING,2010-06-15 08:00:00,2010-07-30 12:00:00,Montana
627,1973,GA,2011-04-29 23:00:00,DR,Severe Storm(s),"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS,...",2011-04-27 18:00:00,2011-04-28 00:00:00,Georgia
1045,2749,TX,2008-03-04 09:00:00,FM,Fire,LA PERLA FIRE,2008-03-03 00:00:00,2008-03-05 00:00:00,Texas
